In [1]:
## General Imports from all libraries
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys, os
import pathlib
import glob
import time
import math, random
import pprint
import collections
import numbers, string

import yaml
from tqdm import tqdm

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk
from PIL import Image

import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

np.set_printoptions(precision=3)
curr_path = pathlib.Path(os.getcwd()).absolute()

cards = !echo $SGE_HGR_gpu_card
device = torch.device(f"cuda:{cards[0]}" if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda:2


In [2]:
# Import custom files for this project
dest_path = str(curr_path.parent.parent)
if dest_path not in sys.path:
    print('Adding current path:', dest_path)
    sys.path.append(str(dest_path))

from run_experiment import batch_metrics

Adding current path: /afs/crc.nd.edu/user/y/yzhang46/_3DPRE/src


# Data Collection and Components

In [3]:
def get_imgs_masks_info(args):
    images_f, mask_f = args
    # 1. Read image and preprocess (clamp + normalize)
    sitk_image = sitk.ReadImage(image_f, sitk.sitkInt16)
    sitk_image = sitk.Clamp(sitk_image, sitk.sitkInt16, -1024, 325)
    sitk_image = sitk.NormalizeImageFilter().Execute(sitk_image)
    image_tensor = torch.from_numpy(sitk.GetArrayFromImage(sitk_image))
    image_tensor = image_tensor.float()

    # 2. Read mask and convert it to one-hot
    sitk_mask = sitk.ReadImage(mask_f, sitk.sitkInt64)
    mask_tensor = torch.from_numpy(sitk.GetArrayFromImage(sitk_mask))

    shape = image_tensor.shape
    oh_shape = [num_classes] + list(shape)
    mask_oh_tensor = torch.zeros(oh_shape, dtype=torch.int32)
    mask_oh_tensor.scatter_(0, mask_tensor.unsqueeze(0), 1)
    
    info = {
        'origin': sitk_image.GetOrigin(),
        'spacing': sitk_image.GetSpacing(),
        'direction': sitk_image.GetDirection()
    }
    return image_tensor, mask_oh_tensor, info
    
    
class MockDataset(torch.utils.data.Dataset):
    def __init__(self, image_files, mask_files, num_classes):
        self.num_classes = num_classes
        self.image_files = image_files  # list of image files
        self.mask_files = mask_files  # list of mask files
        
        # Get image tensors and store permanently
        self.images, self.masks, self.image_info = [], [], []
        args = []
        for image_f, mask_f in zip(image_files, mask_files):
            args.append((image_f, mask_f))
            
        
            
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]
        info_d = self.image_info[idx]
        return image, mask, info_d
        

In [4]:
bcv_dir = pathlib.Path('/afs/crc.nd.edu/user/y/yzhang46/datasets/BCV-2015')
train_image_dir = bcv_dir / 'train' / 'img_nii'
train_mask_dir = bcv_dir / 'train' / 'label_nii'

images = sorted(glob.glob(str(train_image_dir) + '/*.nii.gz'))
masks = sorted(glob.glob(str(train_mask_dir) + '/*.nii.gz'))

start = time.time()
test_size = 6
num_classes = 14
test_set = MockDataset(images[:test_size], masks[:test_size], num_classes)
print(f'Took {time.time() - start} sec to load {test_size} test images.')

Took 19.28532838821411 sec to load 6 test images.


# Model Setup

In [5]:
file = '(0831)bcv-scratch_adam_dice_s3_finetune_bcv_ep499_last.pth'
checkpoint = torch.load(file, map_location='cpu')

from lib.nets.volumetric.resunet3d import UNet3D
# model = UNet3D(1, 14, final_sigmoid=False, is_segmentation=False)

from experiments.finetune_bcv.ftbcv_unet3d import UNet3D as genesis_unet3d
model = genesis_unet3d(n_input=1, n_class=14, act='relu')
model.load_state_dict(checkpoint['state_dict'])

💠 UNet3D-PGL model initiated with n_classes=14, 
   n_input=1, activation=relu, 
   params=19,074,510, trainable_params=19,074,510.


<All keys matched successfully>

# Inference

In [8]:
from data.transforms.crops.inference import ChopBatchAggregate3d as CBA

num_classes = 14
device = 'cuda'
dataset = test_set

model = model.to(device)
model.eval()
with torch.no_grad():
    for i in range(len(dataset)):
        start_vol = time.time()
        print(f'⭐ Inference for example {i+1}..')
        image, mask, info = dataset[i]  # image: DxHxW, mask: CxDxHxW
                                        #  image: float32, mask: int32
        
        # Create Chop, batch, aggregator object
        image = image   # faster calc with GPUs
        mask = mask
        cba = CBA(image, (48, 160, 160), (0, 0, 0), 4, num_classes)
        
        # Run inference on batches of crops of image
        for bidx, batch in enumerate(cba):
            print(f'Batch {bidx + 1} / {len(cba)}')
            crops, locations = batch
            crops = crops.to(device)
            
            logits = model(crops)['out']
    
            cba.add_batch_predictions(logits.cpu(), locations, act='none')
                # NOTE: in this case, we are averaging logits, if you want
                #  to average probabilities instead, use act='softmax'
        
        # Get final predictions, calculate metrics
        agg_predictions = cba.aggregate(ret='one_hot', cpu=True, numpy=False)
        
        print(f'Getting image metrics..')
        start = time.time()
        mets = batch_metrics(agg_predictions.unsqueeze(0), mask.unsqueeze(0))
            # preds are CxDxHxW, but batch input takes 1xCxDxHxW
        print(mets['dice_mean'], mets['dice_class'])
        print(mets['jaccard_mean'], mets['jaccard_class'])
        print(f'Mets time: {time.time() - start:.2f}')
        
        # Convert from 1hot to id and save prediction volume
        print(f'Saving image..')
        id_preds = agg_predictions.argmax(0).numpy().astype(np.uint16)
        sitk_pred = sitk.GetImageFromArray(id_preds, isVector=False)
        sitk_pred.SetOrigin(info['origin'])
        sitk_pred.SetSpacing(info['spacing'])
        sitk_pred.SetDirection(info['direction'])
        sitk.WriteImage(sitk_pred, 'prediction.nii.gz')
        
        elapsed = time.time() - start_vol
        print(f'Completed inference for test volume {i+1} ({elapsed:.2f}).')
        
        break

⭐ Inference for example 1..
Batch 1 / 16
Batch 2 / 16
Batch 3 / 16
Batch 4 / 16
Batch 5 / 16
Batch 6 / 16
Batch 7 / 16
Batch 8 / 16
Batch 9 / 16
Batch 10 / 16
Batch 11 / 16
Batch 12 / 16
Batch 13 / 16
Batch 14 / 16
Batch 15 / 16
Batch 16 / 16
Aggregate (divide):  3.136298656463623
Aggregate:  14.577434301376343
Getting image metrics..
0.06910467892885208 [0.967 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.   ]
0.0669272169470787 [0.937 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.   ]
Mets time: 18.30
Saving image..
Completed inference for test volume 1 (82.26).


In [17]:
logits.argmax(1).unique()

tensor([0], device='cuda:0')